In [1]:
import random
import numpy as np
import math
import time

from numba import jit,prange

from helper_functions import *
from deletion_functions import *
from inference_metaheuristics import *

import matplotlib
import matplotlib.pyplot as plt

from tqdm import tnrange, tqdm_notebook
import multiprocessing as mp
from itertools import repeat

import seaborn as sns
sns.set()
sns.set_style('whitegrid')

In [2]:
def one_iter(N,A,delta, method = None, method_params = None):
    
    X = randseq_uniform(N,A)
    Y = dc(X,delta)
    
    if method == 'exact_ml':
        Xhat = exact_ml(N,A,Y,lambda_forward,delta)
    elif method == 'symbolwise_map':
        Xhat = symbolwise_map(method_params['P_init'],Y,lambda_grad,delta)
    elif method == 'cood_refinement_greedy':
        Xhat = cood_refinement_greedy(method_params['P_init'],Y,\
                                      lambda_grad,delta)    
    elif method == 'cood_refinement_greedy_Y_init':
        start_point = np.pad(Y,(0,N-len(Y)),'constant',constant_values = (0,0))
        method_params['P_init'] = make_categorical(start_point,A)
        Xhat = cood_refinement_greedy(method_params['P_init'],Y,\
                                      lambda_grad,delta)    
    elif method == 'proj_grad_asc':
        Xhat = proj_grad_asc(method_params['P_init'],Y,lambda_grad,\
                             delta,step_size = 0.1,\
                             tolerance = 1e-6,max_grad_steps = 100)    
    else:
        raise ValueError('Method not implemented')
    
#     print(X,Xhat)
    return (hamming_error_rate(Xhat,X),log_likelihood_gain(Xhat,X,Y,delta,A),edit_dist(Xhat,X))

In [3]:
######## Warming up the functions so all jitted codes are compiled #########

N = 10
A = 2
delta = 0.2

method = 'symbolwise_map'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
print(one_iter(N,A,delta, method, method_params))

method = 'exact_ml'
method_params = {}
method_params['P'] = 1/A * np.ones((N,A))
print(one_iter(N,A,delta, method, method_params))

method = 'cood_refinement_greedy'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
print(one_iter(N,A,delta, method, method_params))

method = 'cood_refinement_greedy_Y_init'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
print(one_iter(N,A,delta, method, method_params))

method = 'proj_grad_asc'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
print(one_iter(N,A,delta, method, method_params))

(0.3, 0.0, 0.3)
(0.4, 2.7080502011022105, 0.3)
(0.0, 0.0, 0.0)
(0.3, 1.38629436111989, 0.2)
(0.3, 1.7917594692280554, 0.3)


In [ ]:
### time test for one iteration ####
# for low blocklength use magin function %timeit #
# for large blocklength check how long this bloack takes to run #

N = 100
A = 2
delta = 0.1

method = 'cood_refinement_greedy'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,delta, method, method_params)

In [4]:
def gen_error_rates(N,A,delta_vec, method = None, method_params = None, hyperiters = 100,process_per_hyperiter = 100):
    
    results = {}
    results['summary'] = ("Hamming error rates and likelihood gains for a blocklength of {}, "
    "an alphabet size {} using the method {}".format(N,A,method))
    
    results['delta_vec'] = delta_vec
    
    hamming_error_list = np.zeros((len(delta_vec),hyperiters*process_per_hyperiter))
    likelihood_gain_list = np.zeros((len(delta_vec),hyperiters*process_per_hyperiter))
    
    for idx, delta in enumerate(delta_vec):
#         print('Computing for delta = ',delta)
        time.sleep(0.4)
        pool = mp.Pool(mp.cpu_count())
        for it in tnrange(hyperiters):
            temp = pool.starmap(one_iter, zip(repeat(N),repeat(A),delta*np.ones(process_per_hyperiter),\
                                              repeat(method),repeat(method_params)))
            temp = np.array(temp)
            hamming_error_list[idx,it*process_per_hyperiter:(it+1)*process_per_hyperiter] = temp[:,0]
            likelihood_gain_list[idx,it*process_per_hyperiter:(it+1)*process_per_hyperiter] = temp[:,1]
        pool.close()
    
    results['hamming_error_list'] = hamming_error_list
    results['likelihood_gain_list'] = likelihood_gain_list
    
    return results

In [ ]:
import warnings
warnings.filterwarnings('ignore')

N = 20
A = 2
delta = np.arange(0.1,1,0.1)

hyperiters = 100
process_per_hyperiter = 40

errors = {}

methods = ['cood_refinement_greedy','cood_refinement_greedy_Y_init']

for method in methods:
    print('*'*50,'\n',method,'\n','*'*50)
    method_params = {}
    method_params['P_init'] = 1/A * np.ones((N,A))
    errors[method] = gen_error_rates(N,A,delta, method, method_params,hyperiters,process_per_hyperiter)

In [ ]:
fig = plt.figure(figsize = (6,5))

key = 'cood_refinement_greedy'
plt.plot(errors[key]['delta_vec'],errors[key]['hamming_error_list'].mean(axis = 1),label = 'uniform initialization')

key = 'cood_refinement_greedy_Y_init'
plt.plot(errors[key]['delta_vec'],errors[key]['hamming_error_list'].mean(axis = 1),label = 'initialization based on Y')

plt.legend()
plt.show()

fig.savefig('cood_ref_Y_init.eps',format = 'eps')

In [ ]:
delta = 4
data = []
for key in errors.keys():
    data.append(np.maximum(errors[key]['likelihood_gain_list'][delta],-1))
for key in errors.keys():
    sns.boxplot(data = data)
plt.xlabel(list(errors.keys()))
plt.show()

In [ ]:
np.save('errors_BL100.npy',errors)

In [ ]:
data = []
for key in errors.keys():
    data.append(errors[key]['hamming_error_list'][5])

In [ ]:
for key in errors.keys():
#     plt.plot(errors[key]['delta_vec'],errors[key]['hamming_error_list'].mean(axis = 1))
    sns.violinplot(data = data)

plt.xlabel(errors.keys())
# sns.violinplot(data = result['hamming_error_list'].T)


In [ ]:
# np.save('ML_error.npy',ML_error)
# np.save('MAP_error.npy',MAP_error)
# np.save('ML_cood_error.npy',ML_cood_error)
# np.save('MAP_cood_error.npy',MAP_cood_error)
# np.save('cood_switch_error.npy',cood_switch_error)
# np.save('cood_switch_vertex_error.npy',cood_switch_vertex_error)

In [ ]:
ML_error = np.load('ML_error.npy').item()
MAP_error = np.load('MAP_error.npy').item()
ML_cood_error = np.load('ML_cood_error.npy').item()
MAP_cood_error = np.load('MAP_cood_error.npy').item()
cood_switch_error = np.load('cood_switch_error.npy').item()
cood_switch_vertex_error = np.load('cood_switch_vertex_error.npy').item()

In [ ]:

fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)

ax.plot(MAP_error['del_probs'],MAP_error['hamming_dist'],\
        label = 'Symbolwise MAP', marker = 's', markersize = 10, linewidth = 2)
ax.plot(ML_error['del_probs'],ML_error['hamming_dist'],\
        label = 'ML via gradient ascent', marker = 'o', markersize = 10, linewidth = 2)
ax.plot(ML_cood_error['del_probs'],ML_cood_error['hamming_dist'],\
        label = 'ML grad. asc. + Cood. ref.', marker = 'd', markersize = 10, linewidth = 2)
ax.plot(MAP_cood_error['del_probs'],MAP_cood_error['hamming_dist'],\
        label = 'Sym. MAP + Cood. ref.', marker = '>', markersize = 10, linewidth = 2)
ax.plot(cood_switch_error['del_probs'],cood_switch_error['hamming_dist'],\
        label = 'Coordinate refinement', marker = '^', markersize = 10, linewidth = 2)

ax.plot(cood_switch_vertex_error['del_probs'],cood_switch_vertex_error['hamming_dist'],\
        label = 'Cood. ref. vertex init', marker = 'v', markersize = 10, linewidth = 2)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_xlabel('Deletion Probability',fontsize = 18)
ax.set_ylabel('Hamming error rate',fontsize = 18)
lgd = ax.legend(bbox_to_anchor=(1, -0.15), loc='upper right', ncol=2,fontsize='xx-large')

# fig.savefig('error_rates.eps',format = 'eps', bbox_inches='tight')
plt.show()